# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Experiments import execute_circuits

# Tests

### Without measurement error correction

In [ ]:
qec_cycle_count_list = [1,2,4,8,16,32]
lqc_list = []
qec_cycle_count_list

In [ ]:
lqc_base = LogicalCircuit(1, **steane_code)
lqc_base.encode([0], max_iterations=1)

In [ ]:
lqc_list = []

for n_qec_cycles in qec_cycle_count_list:
    lqc = lqc_base.copy() #copy.deepcopy(lqc_base)
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all(with_error_correction=False)
    lqc_list.append(lqc)

#lqc_list[3].draw("mpl")

In [ ]:
shots = 1E3
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum['H1-1'], coupling_map=None,
  method="statevector", shots=shots
)

In [ ]:
# Option 1: Logical Counts
logical_counts_list = [lqc.get_logical_counts(result.get_counts()) for lqc, result in zip(lqc_list, results)]
wo_qec_infidelities = [logical_counts.get("1", 0)/shots for logical_counts in logical_counts_list] # without qec

# Option 2: LogicalStatevector
# infidelities = []
# for result in results:
#     lsv = LogicalStatevector.from_counts(result.get_counts(), n_logical_qubits=1, **steane_code)
#     infidelity = 1-(lsv.logical_decomposition[0])**2
#     infidelities.append(infidelity)

In [ ]:
fig, ax = plt.subplots(dpi=128)

ax.bar(qec_cycle_count_list, wo_qec_infidelities)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
# ax.set_yscale("log")

plt.show()

### With measurement error correction

In [ ]:
lqc_list = []

for n_qec_cycles in qec_cycle_count_list:
    lqc = lqc_base.copy() #copy.deepcopy(lqc_base)
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all(with_error_correction=True)
    lqc_list.append(lqc)

#lqc_list[3].draw("mpl")

In [ ]:
shots = 1E3
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum['H1-1'], coupling_map=None,
  method="statevector", shots=shots
)

In [ ]:
# Option 1: Logical Counts
logical_counts_list = [lqc.get_logical_counts(result.get_counts()) for lqc, result in zip(lqc_list, results)]
w_qec_infidelities = [logical_counts.get("1", 0)/shots for logical_counts in logical_counts_list] # with qec

# Option 2: LogicalStatevector
# infidelities = []
# for result in results:
#     lsv = LogicalStatevector.from_counts(result.get_counts(), n_logical_qubits=1, **steane_code)
#     infidelity = 1-(lsv.logical_decomposition[0])**2
#     infidelities.append(infidelity)

In [ ]:
fig, ax = plt.subplots(dpi=128)

ax.bar(qec_cycle_count_list, w_qec_infidelities)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
# ax.set_yscale("log")

plt.show()

### Plotting with measurement error correction vs. without

In [ ]:
# Set the x-axis positions for the bars
x_indices = np.arange(len(qec_cycle_count_list))
bar_width = 0.35  # Width of each bar

# Create the figure and axis objects
fig, ax = plt.subplots(figsize=(14, 7))

# Plot the bars for "With QEC" (Blue)
rects1 = ax.bar(x_indices - bar_width/2, w_qec_infidelities, bar_width,
                label='With QEC', color='blue')

# Plot the bars for "Without QEC" (Orange)
rects2 = ax.bar(x_indices + bar_width/2, wo_qec_infidelities, bar_width,
                label='Without QEC', color='orange')

ax.set_ylabel('Infidelity', fontsize=12)
ax.set_title('Comparison of Infidelity With and Without QEC', fontsize=16)
ax.set_xticks(x_indices)
ax.set_xticklabels(qec_cycle_count_list, rotation=45, ha='right')
ax.legend()

# Add grid
ax.yaxis.grid(True, linestyle='--', alpha=0.7)

# Add value labels on top of each bar
def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',  # Format to 3 decimal places
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=8)

# Call the autolabel function for both sets of bars
autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

print("Generating bar plot...")
plt.show()
print("Plot display finished.")


We can test that .copy() produces a deepcopy, rather than a shallow copy:

In [ ]:
lqc_base = LogicalCircuit(1, **steane_code)
lqc_base.encode([0], max_iterations=1)
copy_base = lqc_base.copy()
dir(copy_base)


print(f"Number of qubits in lqc_base: {lqc_base.n_logical_qubits}")
print(f"Number of qubits in copy_base: {copy_base.n_logical_qubits}")

lqc_base.n_logical_qubits = 10000

print(f"Number of qubits in lqc_base: {lqc_base.n_logical_qubits}")
print(f"Number of qubits in copy_base: {copy_base.n_logical_qubits}")

### Quantum circuit baseline

In [ ]:
shots = 1E7
from qiskit import QuantumCircuit
qc = QuantumCircuit(1)
qc.measure_all()
results = execute_circuits(
  qc,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum["H1-1"], coupling_map=None,
  method="statevector", shots=shots
)
counts = results[0].get_counts()
error_rate = counts['1'] / shots
print(f"Error rate: {error_rate}")

### Expr test

In [ ]:
from qiskit.circuit.classical import expr

def cbit_and(cbits, values):
    result = expr.bit_not(cbits[0]) if values[0] == 0 else expr.lift(cbits[0])
    for n in range(len(cbits)-1):
        result = expr.bit_and(result, expr.bit_not(cbits[n+1])) if values[n+1] == 0 else expr.bit_and(result, cbits[n+1])
    return result

cbit_and([0,0,0], [1,0,0])

### Look at circuit structure

In [ ]:
n_qec_cycles = 1
lqc = lqc_base.copy() #copy.deepcopy(lqc_base)
for _ in range(n_qec_cycles):
    lqc.append_qec_cycle()
lqc.measure_all()
#lqc_list[3].draw("mpl")
lqc.decompose().draw(output='mpl')

## LogicalGeneral Testing

In [ ]:
qec_cycle_count_list = [1,2,4,8,16,32]#list(np.arange(0, 10, 1)) #+ list(np.arange(11, 64, 2)) #[1, 2, 4, 8, 16, 32, 64]#np.arange(10) #[0, 1, 2, 3, 4, 5, 6]#, 8, 16, 32]#, 64]
lqc_list = []
qec_cycle_count_list

from LogicalQ.LogicalGeneral import LogicalCircuitGeneral

lqc_base = LogicalCircuitGeneral(1, **steane_code)
lqc_base.encode([0], max_iterations=1)

In [ ]:
for n_qec_cycles in qec_cycle_count_list:
    lqc = lqc_base.copy() #copy.deepcopy(lqc_base)
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all()
    lqc_list.append(lqc)

In [ ]:
shots = 1E3
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum['H1-1'], coupling_map=None,
  method="statevector", shots=shots
)

In [ ]:
logical_counts_list = [lqc.get_logical_counts(result.get_counts()) for lqc, result in zip(lqc_list, results)]
infidelities = [logical_counts.get("1", 0)/shots for logical_counts in logical_counts_list]

fig, ax = plt.subplots(dpi=128)

ax.bar(qec_cycle_count_list, infidelities)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
# ax.set_yscale("log")

plt.show()